<a href="https://www.kaggle.com/mickaelnarboni/seattle-co2-exploration-2016?scriptVersionId=85223950" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Context

The purpose of this notebook is preparing our data to predictive models comparison by doing a short exploration. 

We understand that the goal of the project is about predicting the EnergySTARScore. To do so, we'll have to draw some ideas for our predictive models.

About the cleaning, we can drop the lines that have NaN on their EnergySTARScore because it might biased our predictive model if we impute those data as this is the main variable we want to describe in this project. 

# Glossary

**PropertyGFATotal:** Gross floor area (GFA) - The total floor area contained within the property measured to the external face of the external walls.

**PropertyGFAParking:** Same as PropertyGFATotal but calculated for the parking lot. 

**PropertyGFABuilding(s):** Same as PropertyGFATotal but calculated for each building within the property.

**ENERGYSTARScore:** The ENERGY STAR score provides a comprehensive snapshot of your building’s energy performance, taking into account the building’s physical assets, operations, and occupant behavior. It is expressed on an easy-to-understand 1 to 100 scale, where the higher the score, the better the energy performance of the building. It’ll help you identify which buildings in your portfolio to target for improvement or recognition.

**SiteEUI(kBtu/sf):** EUI stands for Energy Use Intensity. It is simply the energy used by the building at it’s building site and not the power plant. EUI is expressed as energy per square foot per year. It’s calculated by dividing the total energy consumed by the building in one year (measured in kBtu or GJ) by the total gross floor area of the building (measured in square feet or square meters).

**SourceEUI(kBtu/sf):** Source energy refers to a building’s energy footprint since in addition to the site energy, it also accounts for the energy lost during production, transmission, and delivery to the site. Source energy includes the site energy plus all of the energy used to provide and distribute the site energy. As stated before, it includes the entire chain of energy so it is often called Total Energy.

**SiteEnergyUse(kBtu):** Site energy is the energy which is consumed at the final destination of the power generation cycle, and to simplify, is the amount of energy shown on a utility bill. It is the power that is used by the customer, whether residential, commercial, or industrial.

**Electricity(kWh):** Energy consumption in electricity (kWh) for the property

**Electricity(kBtu):** Energy consumption in electricity (kBtu) for the property

**NaturalGas(therms):** Energy consumption in natural gas (therms) for the property

**NaturalGas(kBtu):** Energy consumption in natural gas (kBtu) for the property

**TotalGHGEmissions:** Stands for Total Greenhouse Gas Emissions. It represents the sum of emissions of various gases: carbon dioxide, methane, nitrous oxide, and smaller trace gases such as hydrofluorocarbons (HFCs) and sulfur hexafluoride (SF6).

**GHGEmissionsIntensity:** Greenhouse Gas Emissions intensity (g CO2e/kWh) is calculated as the ratio of CO2e emissions from public electricity production (as a share of CO2 equivalent emissions from public electricity and heat production related to electricity production), and gross electricity production.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore") # ignore the warnings about file size
import matplotlib.pyplot as plt
from matplotlib import colors
%matplotlib inline
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
df = pd.read_csv('../input/sea-building-energy-benchmarking/archive/2016-building-energy-benchmarking.csv')

In [ ]:
df.shape

In [ ]:
# set option to be able to get each column and row of our dataframes
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [ ]:
df.head(10)

In [ ]:
df.describe()

In [ ]:
print('number of columns:',len(df.columns)) # return the number of columns 
print('number of rows:',len(df)) # return the number of rows
print('number of cells:',len(df)*len(df.columns))# return the number of data 
print('number of missing values:',df.isna().sum().sum()) # return the number of total missing values
print('number of missing values in % : {:.2%}'.format((df.isna().sum().sum())/(len(df)*len(df.columns)))) # return the percentage of missing values




In [ ]:
for i in df.columns:
    print(i,'||',df[i].dtypes,'||', df[i].isna().sum(),'||','{:.2%}'.format((df.isna().sum()/len(df))[i])) 

In [ ]:
# split our variables into categorical and continuous types

# categorical variables
df_categorical = pd.DataFrame(data=df.select_dtypes(include=['object']))
df_categorical.head(10)


In [ ]:
df_quantitative = pd.DataFrame(data=df.select_dtypes(include=['int64','float64']))
df_quantitative.head(10)

# Remove Rows with Missing Values

We decide to remobe the rows with missing values on ENERGYSTARScore, because this feature is the one that is central in our project and we don't want to eventually bias our predictions by including missing data for that variable.

In [ ]:
df.isna().sum()

In [ ]:
missing_data = pd.DataFrame()
missing_data['Column variable'] = df.columns
missing_data['Data type'] = list(df.dtypes)
missing_data['Missing values count'] = list(df.isna().sum())
missing_data['% Missing values'] = list((df.isna().sum()/len(df))) 
print(missing_data)

In [ ]:
# data visualization for missing values of our dataframe called missing_data

to_plot = missing_data.sort_values(by=['% Missing values'])
fig, ax = plt.subplots(figsize=(18, 9))
ax.set_facecolor("#2E2E2E")
plt.title('Representation of the % Missing values per column', fontsize=20, y=1.03)
plt.xlabel('Columns Name', fontsize=14)
plt.ylabel('% Missing values', fontsize=14)
plt.grid(True, color="#93a1a1", alpha=0.05)
plt.xticks(rotation=90) # this line allow us to rotate the x_labels so they can be readable
plt.plot(to_plot['Column variable'],to_plot['% Missing values'],color="#E8FF41"); 


In [ ]:
# Based on the above data visualization, we drop the columns and axis with too many NaN values
data = df.drop(['SecondLargestPropertyUseType', 'SecondLargestPropertyUseTypeGFA','ThirdLargestPropertyUseType','ThirdLargestPropertyUseTypeGFA','YearsENERGYSTARCertified','Comments','Outlier'], axis=1)
data = data.dropna(axis = 0, how ='any',subset=['ENERGYSTARScore','ZipCode','LargestPropertyUseType','LargestPropertyUseTypeGFA','SiteEUIWN(kBtu/sf)'])
data.shape

In [ ]:
data.isna().sum().sum()

In [ ]:
pd.crosstab(data['Neighborhood'], data['PrimaryPropertyType'])

In [ ]:
pd.crosstab(data['BuildingType'], data['Neighborhood'])

In [ ]:
fig, ax = plt.subplots(figsize=(16, 16))
plt.title("Distribution of EnergyStarScore varying by Building Type", size=18, y=1.01)
sns.histplot(data=data, x=data['ENERGYSTARScore'], hue=data['BuildingType'], multiple="stack", element="bars", discrete=True, stat="count", kde=True)
ax.set_xlim(0,100)
ax.set_xticks(range(1,100))
plt.xticks(rotation=90)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 16))
plt.title("Distribution of EnergyStarScore varying by Primary Property Type", size=18, y=1.01)
sns.histplot(data=data, x=data['ENERGYSTARScore'], hue=data['PrimaryPropertyType'], multiple="stack", element="bars", discrete=True, stat="count", kde=True)
ax.set_xlim(0,100)
ax.set_xticks(range(1,100))
plt.xticks(rotation=90)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 16))
plt.title("Distribution of EnergyStarScore varying by Neighborhood", size=18, y=1.01)
sns.histplot(data=data, x=data['ENERGYSTARScore'], hue=data['Neighborhood'], multiple="stack", element="bars", discrete=True, stat="count", kde=True)
ax.set_xlim(0,100)
ax.set_xticks(range(1,100))
plt.xticks(rotation=90)
plt.show()

In [ ]:
data.groupby('BuildingType')['ENERGYSTARScore'].describe()

In [ ]:
data.groupby('PrimaryPropertyType')['ENERGYSTARScore'].describe()

In [ ]:
data.groupby('Neighborhood')['ENERGYSTARScore'].describe()

In [ ]:
# we do a scatterplot with x is the longitude and y the latitude, we will use a categorical variable as hue to see if there is any patterns
fig, ax = plt.subplots(figsize=(10, 10))
sns.set_style('darkgrid')
sns.scatterplot(x=data['Longitude'], y=data['Latitude'],hue=data['BuildingType'], palette="bright")
plt.xticks(rotation=90)

In [ ]:
# Univariate Analysis 

#Distribution using boxplot

sns.boxplot(data['ENERGYSTARScore'],color="blue")
plt.xlabel("ENERGYSTARScore", size=16)
plt.title("Distribution of ENERGYSTARScore", size=18, y=1.03)
plt.xlim([-5, 105])
plt.show()

In [ ]:
# Univariate Analysis 

#Distribution using boxplot

sns.boxplot(data['PropertyGFATotal'],color="black")
plt.xlabel("PropertyGFATotal", size=16)
plt.title("Distribution of PropertyGFATotal", size=18, y=1.03)
plt.show()

In [ ]:
# Univariate Analysis 

#Distribution using boxplot

sns.boxplot(data['SiteEUI(kBtu/sf)'],color="green")
plt.xlabel("SiteEUI(kBtu/sf)", size=16)
plt.title("Distribution of SiteEUI(kBtu/sf)", size=18, y=1.03)
plt.xlim([-10, 300])
plt.show()

In [ ]:
# Univariate Analysis 

#Distribution using boxplot

sns.boxplot(data['SourceEUI(kBtu/sf)'],color="pink")
plt.xlabel("SourceEUI(kBtu/sf)", size=16)
plt.title("Distribution of SourceEUI(kBtu/sf)", size=18, y=1.03)
plt.xlim([-10, 300])
plt.show()

In [ ]:
# Univariate Analysis 

#Distribution using boxplot

sns.boxplot(data['Electricity(kWh)'],color="purple")
plt.xlabel("Electricity(kWh)", size=16)
plt.title("Distribution of Electricity(kWh)", size=18, y=1.03)
plt.xlim([-10, 0.5*1e7])
plt.show()

In [ ]:
# Univariate Analysis 

#Distribution using boxplot

sns.boxplot(data['TotalGHGEmissions'],color="red")
plt.xlabel("TotalGHGEmissions", size=16)
plt.title("Distribution of TotalGHGEmissions", size=18, y=1.03)
plt.xlim([-10, 300])
plt.show()

In [ ]:
# Univariate Analysis 

#Distribution using boxplot

sns.boxplot(data['GHGEmissionsIntensity'],color="orange")
plt.xlabel("GHGEmissionsIntensity", size=16)
plt.title("Distribution of TotalGHGEmissions", size=18, y=1.03)
plt.xlim([-1, 5])
plt.show()

In [ ]:
# IQR detection and method
Q1=data.quantile(0.25)
Q3=data.quantile(0.75)
IQR=Q3-Q1
lower_bound = Q1-1.5*IQR
upper_bound = Q3+1.5*IQR
print('Lower bound for each quantitative variable:')
print(lower_bound) 
print('Upper bound for each quantitative variable:')
print(upper_bound) 
# we eliminate all the values above it for each column using IQR, we can notice them with the max() function
data_out = data[~((data<(Q1-1.5*IQR)) | (data>(Q3+1.5*IQR)))]
data_out.describe()

In [ ]:
# fill NaN values with median for relevant columns

median0 = data_out["PropertyGFATotal"].median()
data_out["PropertyGFATotal"].fillna(value = median0,inplace = True)

median1 = data_out["SiteEUI(kBtu/sf)"].median()
data_out["SiteEUI(kBtu/sf)"].fillna(value = median1,inplace = True)

median2 = data_out["SourceEUI(kBtu/sf)"].median()
data_out["SourceEUI(kBtu/sf)"].fillna(value = median2,inplace = True)

median3 = data_out["Electricity(kWh)"].median()
data_out["Electricity(kWh)"].fillna(value = median3,inplace = True)

median4 = data_out["TotalGHGEmissions"].median()
data_out["TotalGHGEmissions"].fillna(value = median4,inplace = True)

median5 = data_out["GHGEmissionsIntensity"].median()
data_out["GHGEmissionsIntensity"].fillna(value = median5,inplace = True)

median6 = data_out["SourceEUIWN(kBtu/sf)"].median()
data_out["SourceEUIWN(kBtu/sf)"].fillna(value = median6,inplace = True)

median7 = data_out["SiteEUIWN(kBtu/sf)"].median()
data_out["SiteEUIWN(kBtu/sf)"].fillna(value = median7,inplace = True)

median8 = data_out["SiteEnergyUse(kBtu)"].median()
data_out["SiteEnergyUse(kBtu)"].fillna(value = median8,inplace = True)

median9 = data_out["SiteEnergyUseWN(kBtu)"].median()
data_out["SiteEnergyUseWN(kBtu)"].fillna(value = median9,inplace = True)

median10 = data_out["SteamUse(kBtu)"].median()
data_out["SteamUse(kBtu)"].fillna(value = median10,inplace = True)

median11 = data_out["Electricity(kBtu)"].median()
data_out["Electricity(kBtu)"].fillna(value = median11,inplace = True)

median12 = data_out["NaturalGas(therms)"].median()
data_out["NaturalGas(therms)"].fillna(value = median12,inplace = True)

median13 = data_out["NaturalGas(kBtu)"].median()
data_out["NaturalGas(kBtu)"].fillna(value = median13,inplace = True)


# drop lines with NaN values in the new dataframe

data_out = data.dropna(axis = 0, how ='any',subset=['OSEBuildingID','ZipCode','Latitude','Longitude','NumberofBuildings','NumberofFloors','PropertyGFAParking','PropertyGFABuilding(s)','LargestPropertyUseTypeGFA'])

data_out.describe()

In [ ]:
# Univariate Analysis (without outliers)

#Distribution using boxplot

sns.boxplot(data_out['ENERGYSTARScore'],color="blue")
plt.xlabel("ENERGYSTARScore", size=16)
plt.title("Distribution of ENERGYSTARScore", size=18, y=1.03)
plt.xlim([-5, 105])
plt.show()

#Distribution using displot

sns.displot(data_out['ENERGYSTARScore'].sort_values(ascending=True), kde=True, color="blue")
plt.title("Distribution of the ENERGYSTARScore", size=18, y=1.03)
plt.ylabel("Density of the distribution (Count)", size=16)
plt.xlabel("ENERGYSTARScore", size=16)
plt.show()


In [ ]:
# Univariate Analysis 

#Distribution using boxplot

sns.boxplot(data_out['PropertyGFATotal'],color="black")
plt.xlabel("PropertyGFATotal", size=16)
plt.title("Distribution of PropertyGFATotal", size=18, y=1.03)
plt.show()

#Distribution using displot

sns.displot(data_out['PropertyGFATotal'].sort_values(ascending=True), kde=True, color="black")
plt.title("Distribution of the PropertyGFATotal", size=18, y=1.03)
plt.ylabel("Density of the distribution (Count)", size=16)
plt.xlabel("PropertyGFATotal", size=16)
plt.show()

In [ ]:
# Univariate Analysis 

#Distribution using boxplot

sns.boxplot(data_out['SiteEUI(kBtu/sf)'],color="green")
plt.xlabel("SiteEUI(kBtu/sf)", size=16)
plt.title("Distribution of SiteEUI(kBtu/sf)", size=18, y=1.03)
plt.xlim([-10, 130])
plt.show()

#Distribution using displot

sns.displot(data_out['SiteEUI(kBtu/sf)'].sort_values(ascending=True), kde=True, color="green")
plt.title("Distribution of the SiteEUI(kBtu/sf)", size=18, y=1.03)
plt.ylabel("Density of the distribution (Count)", size=16)
plt.xlabel("SiteEUI(kBtu/sf)", size=16)
plt.show()

In [ ]:
# Univariate Analysis 

#Distribution using boxplot

sns.boxplot(data_out['SourceEUI(kBtu/sf)'],color="pink")
plt.xlabel("SourceEUI(kBtu/sf)", size=16)
plt.title("Distribution of SourceEUI(kBtu/sf)", size=18, y=1.03)
plt.xlim([-10, 300])
plt.show()

#Distribution using displot

sns.displot(data_out['SourceEUI(kBtu/sf)'].sort_values(ascending=True), kde=True, color="pink")
plt.title("Distribution of the SourceEUI(kBtu/sf)", size=18, y=1.03)
plt.ylabel("Density of the distribution (Count)", size=16)
plt.xlabel("SourceEUI(kBtu/sf)", size=16)
plt.show()

In [ ]:
# Univariate Analysis 

#Distribution using boxplot

sns.boxplot(data_out['Electricity(kWh)'],color="purple")
plt.xlabel("Electricity(kWh)", size=16)
plt.title("Distribution of Electricity(kWh)", size=18, y=1.03)
plt.xlim([-0.01e7, 0.5*1e7])
plt.show()

#Distribution using displot

sns.displot(data_out['Electricity(kWh)'].sort_values(ascending=True), kde=True, color="purple")
plt.title("Distribution of the Electricity(kWh)", size=18, y=1.03)
plt.ylabel("Density of the distribution (Count)", size=16)
plt.xlabel("Electricity(kWh)", size=16)
plt.show()

In [ ]:
# Univariate Analysis 

#Distribution using boxplot

sns.boxplot(data_out['TotalGHGEmissions'],color="red")
plt.xlabel("TotalGHGEmissions", size=16)
plt.title("Distribution of TotalGHGEmissions", size=18, y=1.03)
plt.xlim([-10, 300])
plt.show()

#Distribution using displot

sns.displot(data_out['TotalGHGEmissions'].sort_values(ascending=True), kde=True, color="red")
plt.title("Distribution of the TotalGHGEmissions", size=18, y=1.03)
plt.ylabel("Density of the distribution (Count)", size=16)
plt.xlabel("TotalGHGEmissions", size=16)
plt.show()

In [ ]:
# Univariate Analysis 

#Distribution using boxplot

sns.boxplot(data_out['GHGEmissionsIntensity'],color="orange")
plt.xlabel("GHGEmissionsIntensity", size=16)
plt.title("Distribution of TotalGHGEmissions", size=18, y=1.03)
plt.xlim([-1, 5])
plt.show()

#Distribution using displot

sns.displot(data_out['GHGEmissionsIntensity'].sort_values(ascending=True), kde=True, color="orange")
plt.title("GHGEmissionsIntensity", size=18, y=1.03)
plt.ylabel("Density of the distribution (Count)", size=16)
plt.xlabel("GHGEmissionsIntensity", size=16)
plt.show()

In [ ]:
# correlation matrix
fig, ax = plt.subplots(figsize=(24, 24))
corrMatrix = data_out.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()

# ANOVA

*anova_results* table gives us the results of ANOVA between different categorical variables with ENERGYSTARScore variable.
We notice a relation of 10% with PrimaryPropertyType which is the highest.

In [ ]:
X1 = "PrimaryPropertyType" # categorical 1
X2 = "BuildingType" # categorical 2
X3 = "Neighborhood" # categorical 3

Y1 = "ENERGYSTARScore" # quantitative 1 
Y2 = "GHGEmissionsIntensity" # quantitative 2
Y3 = "TotalGHGEmissions" # quantitative 3

sample_anova = data_out[data_out["ENERGYSTARScore"] < 100] # we make sure we do our analysis based on filtered data

def eta_squared(x,y):
    mean_y = y.mean()
    classes = []
    for classe in x.unique():
        yi_classe = y[x==classe]
        classes.append({'ni': len(yi_classe),
                        'mean_classe': yi_classe.mean()})
    SCT = sum([(yj-mean_y)**2 for yj in y])
    SCE = sum([c['ni']*(c['mean_classe']-mean_y)**2 for c in classes])
    return SCE/SCT
    
x1 = eta_squared(sample_anova[X1],sample_anova[Y1]) # PrimaryPropertypeType against ENERGYSTARScore
x2 = eta_squared(sample_anova[X2],sample_anova[Y1]) # BuildingType against ENERGYSTARScore
x3 = eta_squared(sample_anova[X3],sample_anova[Y1]) # Neighborhood against ENERGYSTARScore

x4 = eta_squared(sample_anova[X1],sample_anova[Y2]) # PrimaryPropertypeType against GHGEmissionsIntensity
x5 = eta_squared(sample_anova[X2],sample_anova[Y2]) # BuildingType against GHGEmissionsIntensity
x6 = eta_squared(sample_anova[X3],sample_anova[Y2]) # Neighborhood against GHGEmissionsIntensity

x7 = eta_squared(sample_anova[X1],sample_anova[Y3]) # PrimaryPropertypeType against TotalGHGEmissions
x8 = eta_squared(sample_anova[X2],sample_anova[Y3]) # BuildingType against TotalGHGEmissions
x9 = eta_squared(sample_anova[X3],sample_anova[Y3]) # Neighborhood against TotalGHGEmissions

array1 = np.array([["PrimaryPropertyType",x1],["BuildingType",x2],["Neighborhood",x3]])
array2 = np.array([["PrimaryPropertyType",x4],["BuildingType",x5],["Neighborhood",x6]])
array3 = np.array([["PrimaryPropertyType",x7],["BuildingType",x8],["Neighborhood",x9]])

In [ ]:
anova_results_energystarscore = pd.DataFrame(array1, columns =['Variable','Eta$^{2}$'])
anova_results_energystarscore.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plt.title("Density of ANOVA results for ENERGYSTARScore versus Categorical Variables", size=18, y=1.01)
sns.barplot(x=anova_results_energystarscore["Variable"], y=anova_results_energystarscore["Eta$^{2}$"].astype(np.float))
plt.show()

In [ ]:
anova_results_ghgemissionsintensity = pd.DataFrame(array2, columns =['Variable','Eta**2'])
anova_results_ghgemissionsintensity.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plt.title("Density of ANOVA results for GHGEmissionsIntensity versus Categorical Variables", size=18, y=1.01)
sns.barplot(x=anova_results_ghgemissionsintensity["Variable"], y=anova_results_ghgemissionsintensity["Eta**2"].astype(np.float))
plt.show()

In [ ]:
anova_results_totalghgemissions = pd.DataFrame(array3, columns =['Variable','Eta**2'])
anova_results_totalghgemissions.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plt.title("Density of ANOVA results for TotalGHGEmissions versus Categorical Variables", size=18, y=1.01)
sns.barplot(x=anova_results_totalghgemissions["Variable"], y=anova_results_totalghgemissions["Eta**2"].astype(np.float))
plt.show()

# Preparation for Modeling

In [ ]:
import os
os.chdir(r'./')
data_out.to_csv('clean_p3.csv',sep = '\t',index = True)
from IPython.display import FileLink
FileLink(r'clean_p3.csv')